# Queries with and without Azure OpenAI

지금까지 인덱스의 데이터 소스에서 검색 엔진을 로드했습니다. 이 노트북에서는 몇 가지 예제 쿼리를 시도한 다음 Azure OpenAI 서비스를 사용하여 사용자 쿼리에 대한 올바른 답변을 얻을 수 있는지 살펴보겠습니다.

사용자가 콘토소 일렉트로닉스 HR 문서에 대해 질문하면 엔진이 그에 따라 응답하는 것입니다. 이 단일 인덱스 데모는 회사에서 완전히 다른 주제에 대해 서로 다른 유형의 단일 문서를 로드하고 검색 엔진이 가장 관련성이 높은 결과로 응답해야 하는 시나리오를 모방한 것입니다.



## Set up variables

In [ ]:
import os
import urllib
import requests
import random
import json
from collections import OrderedDict
from IPython.display import display, HTML, Markdown
from typing import List
from operator import itemgetter

# LangChain Imports needed
from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.retrievers import BaseRetriever
from langchain_core.callbacks import CallbackManagerForRetrieverRun
from langchain_core.documents import Document
from langchain_core.runnables import ConfigurableField


# Our own libraries needed
from common.prompts import DOCSEARCH_PROMPT
from common.utils import get_search_results

from dotenv import load_dotenv
load_dotenv("credentials.env")

In [ ]:
# Setup the Payloads header
headers = {'Content-Type': 'application/json','api-key': os.environ['AZURE_SEARCH_KEY']}
params = {'api-version': os.environ['AZURE_SEARCH_API_VERSION']}

## Single-Index Search queries

In [ ]:
# Text-based Indexe that we are going to query (from Notebook 01)
index_name = "cogsrch-index-hrdocs"
indexes = [index_name]

건강 보험, 직원에 대한 직무 설명과 같이 Contoso Electronic의 정책에서 답변하거나 다룰 수 있다고 생각되는 질문을 해보세요. 결과를 ChatGPT의 오픈 버전과 비교해 보세요. <br>

Azure OpenAI를 사용한 답변은 이러한 게시물에 포함된 정보만 살펴본다는 점을 기억하세요.

**Example Questions you can ask**:
- What is our mission?
- How do we review the performance?
- What is the responsibility of Manager of Human Resources?


In [ ]:
QUESTION = "What is the responsibility of Manager of Human Resources?"

### Search on an index

#### **Note**:
멀티 인덱스도 사용할 수 있습니다. 멀티 인덱스를 사용하려면 다음과 같이 하세요. 

표준화 인덱스를 사용해야 합니다. 인덱스를 표준화하려면 **각 인덱스에 6개의 필수 필드가 있어야 합니다**:
6개 필수 필드는 `id, title, name, location, chunk, chunkVector`입니다.

이는 각 문서가 코드 상에서 동일하게 취급될 수 있도록 하기 위함입니다. 또한 **모든 인덱스에는 시맨틱 구성이 있어야 합니다**.

하이브리드 쿼리를 사용하겠습니다. 최적의 결과를 위해 텍스트 + 벡터 검색을 결합한 하이브리드 쿼리를 사용하겠습니다!

In [ ]:
agg_search_results = dict()
k = 10

for index in indexes:
    search_payload = {
        "search": QUESTION, # Text query
        "select": "id, title, name, location, chunk",
        "queryType": "semantic",
        "vectorQueries": [{"text": QUESTION, "fields": "chunkVector", "kind": "text", "k": k}], # Vector query
        "semanticConfiguration": "my-semantic-config",
        "captions": "extractive",
        "answers": "extractive",
        "count":"true",
        "top": k
    }

    r = requests.post(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexes/" + index + "/docs/search",
                     data=json.dumps(search_payload), headers=headers, params=params)
    print(r.status_code)

    search_results = r.json()
    agg_search_results[index]=search_results
    print("Index:", index, "Results Found: {}, Results Returned: {}".format(search_results['@odata.count'], len(search_results['value'])))

### 점수에 따라 상위 검색 결과(검색 결과)를 표시하기 

In [ ]:
display(HTML('<h4>Top Answers</h4>'))

for index,search_results in agg_search_results.items():
    for result in search_results['@search.answers']:
        if result['score'] > 0.5: # Show answers that are at least 50% of the max possible score=1
            display(HTML('<h5>' + 'Answer - score: ' + str(round(result['score'],2)) + '</h5>'))
            display(HTML(result['text']))
            
print("\n\n")
display(HTML('<h4>Top Results</h4>'))

content = dict()
ordered_content = OrderedDict()


for index,search_results in agg_search_results.items():
    for result in search_results['value']:
        if result['@search.rerankerScore'] > 1:# Show answers that are at least 25% of the max possible score=4
            content[result['id']]={
                                    "title": result['title'],
                                    "chunk": result['chunk'], 
                                    "name": result['name'], 
                                    "location": result['location'] ,
                                    "caption": result['@search.captions'][0]['text'],
                                    "score": result['@search.rerankerScore'],
                                    "index": index
                                    }
    
#After results have been filtered we will Sort and add them as an Ordered list\n",
for id in sorted(content, key= lambda x: content[x]["score"], reverse=True):
    ordered_content[id] = content[id]
    url = str(ordered_content[id]['location']) + os.environ['BLOB_SAS_TOKEN']
    title = str(ordered_content[id]['title']) if (ordered_content[id]['title']) else ordered_content[id]['name']
    score = str(round(ordered_content[id]['score'],2))
    display(HTML('<h5><a href="'+ url + '">' + title + '</a> - score: '+ score + '</h5>'))
    display(HTML(ordered_content[id]['caption']))

### 쿼리 결과에 대한 코멘트

위에서 본 바와 같이 Azure AI 검색 서비스의 시맨틱 리랭크 기능은 훌륭합니다. 
때로는 답변과 함께 해당 파일과 답변이 가능한 단락이 있는 상위 결과도 제공합니다.

Azure OpenAI로 이 기능을 더 개선할 수 있는지 살펴봅시다.

# Using Azure OpenAI

OpenAI를 사용하여 질문에 대한 더 나은 답변을 얻기 위한 일련의 사고 과정은 간단합니다.  검색 결과의 답변과 문서 내용을 컨텍스트로 GPT 모델에 제공**하고 더 나은 답변을 제공하도록 하는 것입니다. 이것이 바로 RAG(Retreival Augmented Generation, 검색 증강 생성)입니다.

이제 이 작업을 수행하기 전에 몇 가지 사항을 먼저 이해해야 합니다.

1) Chainning and Prompt Engineering 
2) Embeddings

우리는 많은 상용구 코드를 패키징하는 **LangChain**이라는 라이브러리를 사용할 것입니다.
Langchain은 내부에서 많은 프롬프트 엔지니어링을 수행하는 라이브러리 중 하나이며, 자세한 내용은 [여기](https://python.langchain.com/en/latest/index.html)를 참조하세요.

In [ ]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]

**Important Note**: 지금부터 OpenAI 모델을 활용합니다. Azure OpenAI 포털 내에서 다음 모델을 배포했는지 확인하세요. 

- text-embedding-ada-002 (or newer)
- gpt-35-turbo (1106 or newer)
- gpt-4-turbo (1106 or newer)

Reference for Azure OpenAI models (regions, limits, dimensions, etc): [HERE](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models)

## chaining LLMs과 prompt engineering에 대한 소개

체인은 LLM, 도구 또는 데이터 전처리 단계에 대한 호출 시퀀스를 나타냅니다.

Azure OpenAI는 사용할 수 있는 LLM(공급자)의 한 유형이지만 Cohere, Huggingface 등과 같은 다른 유형도 있습니다.

체인은 단순(예: 일반) 또는 전문화(예: 유틸리티)될 수 있습니다.

일반 - 가장 간단한 체인은 단일 LLM입니다. 입력 프롬프트와 LLM의 이름을 받은 다음 텍스트 생성(즉, 프롬프트에 대한 출력)에 LLM을 사용합니다. 다음은 예시입니다:

In [ ]:
COMPLETION_TOKENS = 2500
llm = AzureChatOpenAI(deployment_name=os.environ["GPT35_DEPLOYMENT_NAME"], 
                      temperature=0, 
                      max_tokens=COMPLETION_TOKENS)

In [ ]:
output_parser = StrOutputParser()
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an assistant that give thorough responses to users."),
    ("user", "{input}. Give your response in {language}")
])

 | 기호는 유닉스 파이프 연산자와 유사하며, 서로 다른 컴포넌트를 연결하여 한 컴포넌트의 출력을 다음 컴포넌트의 입력으로 공급하는 역할을 합니다.

In [ ]:
chain = prompt | llm | output_parser

In [ ]:
%%time
display(Markdown(chain.invoke({"input": QUESTION, "language": "English"})))

**참고**: 이 액셀러레이터에서 OpenAI를 처음 사용하는 경우, 리소스를 찾을 수 없음 오류가 발생하면 OpenAI 모델 배포 이름이 위에 설정된 환경 변수 os.environ["GPT35_DEPLOYMENT_NAME"]과 다르기 때문일 가능성이 높습니다.

이제 간단한 프롬프트를 생성하고 ChatGPT 지식을 사용하여 일반적인 질문에 답하기 위해 체인을 사용하는 방법을 알게 되었습니다!

일반 체인을 독립형 체인으로 사용하는 경우는 거의 없다는 점에 유의하는 것이 중요합니다. 더 자주 유틸리티 체인의 빌딩 블록으로 사용됩니다 (다음에 살펴보겠습니다). 또한 주목해야 할 중요한 점은 아직 문서나 Azure 검색 결과를 사용하는 것이 아니라 학습된 데이터에 대한 ChatGPT의 지식만 사용한다는 것입니다.

**두 번째 유형의 체인은 유틸리티입니다.**

유틸리티 - 특정 작업을 해결하는 데 도움이 되는 여러 빌딩 블록으로 구성된 전문화된 체인입니다. 
<br> 예를 들어, LangChain은 일부 엔드투엔드 체인을 지원합니다(예: QnA 문서 검색, 요약 등을 위한 create_retrieval_chain).

이 워크샵에서는 더 깊이 파고들기 위해 자체적으로 특정 체인을 구축하고 Azure AI Search의 결과를 개선하는 사용 사례를 해결해 보겠습니다.

하지만 필요한 유틸리티 체인을 다루기 전에 먼저 임베딩과 벡터 검색 및 RAG의 개념을 살펴봅시다.

## Embeddings and Vector Search

Azure OpenAI 에서 임베딩은 기계 학습 모델 및 알고리즘에서 쉽게 활용할 수 있는 특별한 형식의 데이터 표현입니다. 임베딩은 텍스트의 의미론적 의미를 밀도 있게 표현한 정보입니다. 각 임베딩은 부동 소수점 숫자로 이루어진 벡터로, 벡터 공간에서 두 임베딩 사이의 거리는 원래 형식의 두 입력 사이의 의미적 유사성과 상관관계가 있습니다. 예를 들어 두 텍스트가 유사하다면 벡터 표현도 유사해야 합니다.



### Why Do We Need Vectors?

벡터는 여러 가지 이유로 필수적입니다:

- **의미론적 풍부함**: 벡터는 텍스트의 의미론적 의미를 수학적 벡터로 변환하여 단순한 키워드 검색으로는 놓칠 수 있는 뉘앙스를 포착합니다. 따라서 언어를 이해하고 처리하는 데 매우 강력합니다.
- **인간과 유사한 검색**: 벡터 거리를 이용한 검색은 정확한 단어 일치에만 의존하지 않고 문맥과 의미에 따라 정보를 찾는 인간의 접근 방식을 모방합니다.
- **규모의 효율성**: 벡터 표현을 사용하면 대규모 데이터 세트를 효율적으로 처리하고 검색할 수 있습니다. 복잡한 텍스트를 숫자 벡터로 줄임으로써 알고리즘은 방대한 양의 정보를 빠르게 선별할 수 있습니다.



### Understanding LLM Tokens' Context Limitation

GPT와 같은 대규모 언어 모델(LLM)에는 각 입력에 대한 토큰 제한이 있어 긴 문서나 광범위한 데이터 세트를 다룰 때 문제가 됩니다. 이러한 제한은 제공된 정보의 전체 맥락에 따라 이해하고 응답을 생성하는 모델의 능력을 제한합니다. 따라서 이러한 제한을 효과적으로 관리하고 우회하여 LLM의 성능을 최대한 활용할 수 있는 전략을 수립하는 것이 매우 중요합니다.

이 문제를 해결하기 위해 이 솔루션에는 몇 가지 주요 단계가 포함되어 있습니다.

1. 문서 세분화: 대용량 문서를 관리하기 쉬운 작은 세그먼트로 세분화합니다. 
2. 청크의 벡터화: 이러한 세그먼트를 벡터로 변환해 벡터 기반 검색 기술과 호환되도록 합니다. 
3. 하이브리드 검색: 벡터 및 텍스트 검색 방법을 모두 사용하여 쿼리와 관련하여 가장 관련성이 높은 세그먼트를 찾아냅니다. 
4. 최적의 컨텍스트 제공: 가장 관련성이 높은 세그먼트를 LLM에 제시하여 토큰 한도 내에서 세부 사항과 간결함 사이의 균형을 유지합니다.

우리의 궁극적인 목표는 벡터 인덱스와 하이브리드 검색(벡터 + 텍스트)에만 의존하는 것입니다. 다양한 파일 형식에 대해 OCR을 사용하여 파서를 수동으로 코딩하고 데이터를 인덱스와 동기화하는 스케줄러를 개발할 수도 있지만, 더 효율적인 대안이 있습니다: 바로 자동화된 청킹 전략과 벡터화를 제공하는 Azure AI Search입니다. 'ordered_content' 사전에서 볼 수 있듯이 문서 세분화 및 벡터화는 이미 AI Azure Search에서 완료되었다는 점에 유의해야 합니다. 이 전처리 단계는 후속 작업을 간소화하여 빠른 응답 시간을 보장하고 선택한 OpenAI 모델의 토큰 한도를 준수합니다.


따라서 이제 우리가 할 일은 Azure AI 검색 쿼리의 결과가 LLM 컨텍스트 크기에 맞는지 확인한 다음 마법이 작동하도록 내버려두는 것뿐입니다.

In [ ]:
index_name = "cogsrch-index-hrdocs"
indexes = [index_name]

코드 중복을 피하기 위해 위에서 사용한 많은 코드를 함수에 넣었습니다. 이러한 함수는 common/utils.py 및 common/prompts.py 파일에 있습니다. 이렇게 하면 나중에 빌드할 앱에서 이러한 함수를 사용할 수 있습니다.

get_search_results()는 다중 인덱스 검색을 수행하고 문서/청크의 결합된 정렬된 목록을 반환합니다.

In [ ]:
k = 10  # play with this parameter and see the quality of the final answer
ordered_results = get_search_results(QUESTION, indexes, k=k, reranker_threshold=1)
print("Number of results:",len(ordered_results))

In [ ]:
top_docs = []
for key,value in ordered_results.items():
    if value["score"] > 3:
        location = value["location"] if value["location"] is not None else ""
        top_docs.append(Document(page_content=value["chunk"], metadata={"source": location, "score":value["score"]}))

print("Number of filtered results:",len(top_docs))

In [ ]:
chain = (
    DOCSEARCH_PROMPT  # Passes the 4 variables above to the prompt template
    | llm   # Passes the finished prompt to the LLM
    | StrOutputParser()  # converts the output (Runnable object) to the desired output (string)
)

answer = chain.invoke({"question": QUESTION, "context":top_docs})

print(answer)

### From GPT-3.5 to GPT-4

이제 GPT-4로 변경하면 응답이 어떻게 바뀌는지 살펴보겠습니다.


In [ ]:
llm_2 = AzureChatOpenAI(deployment_name=os.environ["GPT4_DEPLOYMENT_NAME"], temperature=0.5, max_tokens=COMPLETION_TOKENS)
chain = DOCSEARCH_PROMPT | llm_2 | output_parser

In [ ]:
%%time
try:
    display(Markdown(chain.invoke({"question": QUESTION, "context": top_docs})))
except Exception as e:
    print(e)

#### As we can see, the model selection MATTERS!

이에 대해서는 나중에 자세히 살펴보겠지만, 지금은 **품질과 응답 시간에서 GPT3.5와 GPT4의 차이점을 살펴보겠습니다**.

# 프롬프트 개선 및 인용구 추가

위에서 "사용자에 대한 철저한 대응"이라는 메시지가 표시되어 있음에도 불구하고 GPT3.5의 답변은 GPT4에 비해 매우 단순하다는 것을 알 수 있습니다. 또한 인용이나 참고 문헌이 없다는 것도 확인할 수 있었습니다. **답변이 문맥에 근거한 것인지 아닌지 어떻게 알 수 있을까요?**

프롬프트 엔지니어링으로 이 두 가지 문제를 개선할 수 있는지 살펴봅시다.
`common/prompts.py`에 `DOCSEARCH_PROMPT`라는 프롬프트를 만들었으니 확인해 보세요!


또한 체인 building 내에서 쉽게 연결할 수 있도록 사용자 정의 리트리버 클래스를 만들어 보겠습니다. 
참고: Azure AI 검색 리트리버 클래스 [여기](https://python.langchain.com/docs/integrations/vectorstores/azuresearch)를 사용할 수도 있지만, 다음과 같은 이유로 사용자 지정 리트리버를 만들고자 합니다.


1) 한 번의 호출로 멀티 인덱스 검색 즉 여러 인덱스를 검색을 수행하려고 합니다.
2) 이 노트북에서 LangChain의 복잡한 개념을 더 쉽게 가르칠 수 있습니다.
3) REST API를 사용하려는 경우와 Python Azure Search SDK를 사용하려는 경우의 차이를 이해합니다.

In [ ]:
class CustomRetriever(BaseRetriever):
    
    topK : int
    reranker_threshold : int
    indexes: List
    sas_token: str = None
    
    def _get_relevant_documents(self, query: str) -> List[Document]:
        
        ordered_results = get_search_results(query, self.indexes, k=self.topK, 
                                             reranker_threshold=self.reranker_threshold, 
                                             sas_token=self.sas_token)
        top_docs = []
        for key,value in ordered_results.items():
            location = value["location"] if value["location"] is not None else ""
            top_docs.append(Document(page_content=value["chunk"], metadata={"source": location, "score":value["score"]}))

        return top_docs

In [ ]:
# Create the retriever
retriever = CustomRetriever(indexes=indexes, topK=k, reranker_threshold=1, sas_token=os.environ['BLOB_SAS_TOKEN'])

In [ ]:
# Test retreiver
results = retriever.get_relevant_documents(QUESTION)
len(results)

In [ ]:
# We can create now a dynamically configurable llm object that can change the model at runtime
dynamic_llm = AzureChatOpenAI(deployment_name=os.environ["GPT35_DEPLOYMENT_NAME"], 
                              temperature=0.5, max_tokens=COMPLETION_TOKENS).configurable_alternatives(
    # This gives this field an id
    # When configuring the end runnable, we can then use this id to configure this field
    ConfigurableField(id="model"),
    # This sets a default_key.
    # If we specify this key, the default LLM  (initialized above) will be used
    default_key="gpt35",
    # This adds a new option, with name `gpt4`
    gpt4=AzureChatOpenAI(deployment_name=os.environ["GPT4_DEPLOYMENT_NAME"], 
                         temperature=0.5, max_tokens=COMPLETION_TOKENS),
    # You can add more configuration options here
)

In [ ]:
# Declaration of the chain with the dynamic llm and the new prompt
configurable_chain = (
    {
        "context": itemgetter("question") | retriever, # Passes the question to the retriever and the results are assign to context
        "question": itemgetter("question")
    }
    | DOCSEARCH_PROMPT  # Passes the input variables above to the prompt template
    | dynamic_llm   # Passes the finished prompt to the LLM
    | StrOutputParser()  # converts the output (Runnable object) to the desired output (string)
)

In [ ]:
%%time

try:
    display(Markdown(configurable_chain.with_config(configurable={"model": "gpt35"}).invoke({"question": QUESTION})))
except Exception as e:
    print(e)

위에서 보았듯이 프롬프트 엔지니어링만으로 답변의 품질과 완성도를 높이고 인용을 추가할 수 있었습니다!


다시 GPT-4를 시도해 봅시다.

In [ ]:
%%time
try:
    display(Markdown(configurable_chain.with_config(configurable={"model": "gpt4"}).invoke({"question": QUESTION})))
except Exception as e:
    print(e)

#### 보다시피 GPT4의 답변은 더 풍부하고 모든 관련 청크를 포함합니다. GPT3.5는 첫 번째와 마지막 청크에만 집중하는 경향이 있습니다.

## Adding Streaming to improve user experience and performance

이쯤 되면 **GPT4 답변이 GPT3.5**보다 품질이 더 우수하다는 것을 알 수 있습니다. 틀린 답은 없지만, 문맥을 이해하고 프롬프트 지침을 따르며 포괄적인 답변을 제공하는 데 있어서는 GPT4가 더 우수합니다.

GPT4를 더 빠르게 보이게 하는 한 가지 방법은 사용자가 입력하는 대로 답변을 볼 수 있도록 답변을 스트리밍하는 것입니다. 이렇게 하려면 `invoke` 대신 `stream` 메서드를 호출하기만 하면 됩니다. 나중에 노트북에서 스트리밍과 콜백에 대해 자세히 설명하지만, 지금은 간단한 방법을 소개합니다.

In [ ]:
for chunk in configurable_chain.with_config(configurable={"model": "gpt4"}).stream({"question": QUESTION}):
    print(chunk, end="", flush=True)

# Summary
##### OpenAI를 사용하면 사용자 질문에 대한 답변이 Azure AI Search의 결과만 가져오는 것보다 훨씬 더 나은 결과를 얻을 수 있습니다. 요약하면 다음과 같습니다. 
- Azure AI Search를 활용하여 문서의 상위 chunk를 식별하는 단일 인덱스 하이브리드 검색을 수행합니다.
- 그런 다음, Azure OpenAI는 이러한 추출된 청크를 컨텍스트로 활용하고 콘텐츠를 이해한 후 최적의 답변을 제공하는 데 사용합니다.
- Best of two worlds!

##### Important observations on this notebook:

1) GPT-3.5를 사용한 답변은 품질은 떨어지지만 훨씬 빠릅니다.
2) GPT-3.5를 사용한 답변이 올바른 형식의 인용을 찾지 못하는 경우가 있습니다.
3) GPT-4를 사용한 답변은 품질은 우수하지만 속도가 훨씬 느립니다.
4) GPT-4를 사용한 답변은 항상 올바른 형식의 우수하고 다양한 인용을 제공합니다.
5) 답변을 스트리밍하면 사용자 경험이 크게 향상됩니다!

# NEXT
다음 노트북에서는 벡터 검색을 사용하여 복잡하고 큰 문서를 개별적으로 처리하는 방법을 살펴보겠습니다.